To Do:

I need to create a Syft equivalent of nn.Module that lets you define a model if you initilize it with all the layers it will use, and define its forward pass using the activation functions.

SyMPC seems to make a Module equivalent but for each type of layer- i.e. Conv2d is implemented as a Module, and (I think) their MPCTensor tracks all its gradients.

Perhaps this will come down to modifying the loss function to take the DP Tensor as input, but I think if we use `publish` in order to do the conversion from DP Tensor -> array/tensor, we wouldn't have to do this modification


In [1]:
import syft

/opt/anaconda3/envs/Hagrid/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch

In [3]:
from syft import nn

In [4]:
import syft.core.tensor.nn.functional as F

In [5]:
from typing import Union
from typing import Sequence
from typing import Tuple, Optional
from syft import PhiTensor

class Conv2d(torch.nn.Module):
    
    def __init__(self, in_channels:int, out_channels:int, kernel_size: Union[int, Sequence[int]], padding:int):
        super(Conv2d, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.padding = padding
        
    def forward(self, x: PhiTensor):
        return nn.Conv2d(
            image=x, 
            in_channels=self.in_channels, 
            out_channels=self.out_channels, 
            kernel_size=self.kernel_size, 
            padding=self.padding
        )
    
    
class BatchNorm2d(torch.nn.Module):
    
    def __init__(self, num_features, eps=1e-05, momentum=0.1, affine=True):
        super(BatchNorm2d, self).__init__()
        self.num_features = num_features
        self.eps = eps
        self.momentum = momentum
        self.affine = affine
        
    def forward(self, x: PhiTensor):
        return nn.BatchNorm2d(
            image=x, 
            num_features=self.num_features, 
            eps=self.eps, 
            momentum=self.momentum, 
            affine=self.affine
        )

class MaxPool2d(torch.nn.Module):
    
    def __init__(self, kernel_size: Union[int, Tuple[int, int]],
                 stride: Optional[Union[int, Tuple[int, int]]] = None,
                 padding: Union[int, Tuple[int, int]] = 0,
                 dilation: int = 1,
                 return_indices: bool = False,
                 ceil_mode: bool = False,
                ):
        super(MaxPool2d, self).__init__()
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.return_indices = return_indices
        self.ceil_mode = ceil_mode
        
    def forward(self, x: PhiTensor):
        return nn.MaxPool2d(
            image=x, 
            kernel_size=self.kernel_size, 
            stride=self.stride, 
            padding=self.padding, 
            return_indices=self.return_indices, 
            ceil_mode=self.ceil_mode
        )
    
class AvgPool2d(torch.nn.Module):
    def __init__(
        self, 
        kernel_size: Union[int, Tuple[int, int]], 
        stride: Optional[Union[int, Tuple[int, int]]] = None, 
        padding: Union[int, Tuple[int, int]] = 0
    ):
        super(AvgPool2d, self).__init__()
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        
    def forward(self, x: PhiTensor):
        return nn.AvgPool2d(image=x, kernel_size=self.kernel_size, stride=self.stride, padding=self.padding)
    
class Linear(torch.nn.Module):
    def __init__(self, in_features: int, out_features: int, bias: bool = True):
        super(Linear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.bias = bias
    
    def forward(self, x: PhiTensor):
        return nn.Linear(image=x, in_features=self.in_features, out_features=self.out_features, bias=self.bias)

In [6]:
class ConvNet(torch.nn.Module):
    
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2)
        self.conv2 = Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=2)
        self.conv3 = Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=2)
        self.conv4 = Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=2)
        self.conv5 = Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=2)
        self.bn1 = BatchNorm2d(32)
        self.bn2 = BatchNorm2d(64)
        self.bn3 = BatchNorm2d(128)
        self.bn4 = BatchNorm2d(256)
        self.bn5 = BatchNorm2d(512)
        self.pool = MaxPool2d(kernel_size=2, stride=2)
        self.avg = AvgPool2d(3)
        self.fc = Linear(512 * 1 * 1, 2)
        
    def forward(self, x: PhiTensor):
        # First layer of CNN - running 1 at a time to debug and see if any individual componenet is failing
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.leaky_relu(x)
        x = self.pool(x)
        
        # Subsequent layers
        x = self.pool(F.leaky_relu(self.bn2(self.conv2(x))))
        x = self.pool(F.leaky_relu(self.bn3(self.conv3(x))))
        x = self.pool(F.leaky_relu(self.bn4(self.conv4(x))))
        x = self.pool(F.leaky_relu(self.bn5(self.conv5(x))))
        x = self.avg(x)
        x = x.view(-1, 512 * 1 * 1) # !!!
        x = self.fc(x)
        return x

In [7]:
cnn_model = ConvNet()

In [8]:
cnn_model

ConvNet(
  (conv1): Conv2d()
  (conv2): Conv2d()
  (conv3): Conv2d()
  (conv4): Conv2d()
  (conv5): Conv2d()
  (bn1): BatchNorm2d()
  (bn2): BatchNorm2d()
  (bn3): BatchNorm2d()
  (bn4): BatchNorm2d()
  (bn5): BatchNorm2d()
  (pool): MaxPool2d()
  (avg): AvgPool2d()
  (fc): Linear()
)

In [9]:
from syft import PhiTensor
import numpy as np

N = 10
C_in = 3
H_in = 50
W_in = 50

input_shape = (N, C_in, H_in, W_in)
x = PhiTensor(child=np.random.randint(low=0, high=255, size=input_shape),
              data_subjects=np.zeros(input_shape),
              min_vals=0,
              max_vals=255
             )

In [19]:
prediction = cnn_model(x)

In [20]:
prediction.min_vals

<lazyrepeatarray data: [[0.05091886 0.01344587]
 [0.05091885 0.01344587]
 [0.05091886 0.01344587]
 [0.05091885 0.01344587]
 [0.05091886 0.01344587]
 [0.05091885 0.01344587]
 [0.05091886 0.01344587]
 [0.05091885 0.01344587]
 [0.05091886 0.01344587]
 [0.05091885 0.01344587]] -> shape: (10, 2)>

In [21]:
prediction.max_vals

<lazyrepeatarray data: [[-1.0008134 -1.4859134]
 [-1.0008131 -1.4859128]
 [-1.0008134 -1.4859134]
 [-1.0008131 -1.4859128]
 [-1.0008134 -1.4859134]
 [-1.0008131 -1.4859128]
 [-1.0008134 -1.4859134]
 [-1.0008131 -1.4859128]
 [-1.0008134 -1.4859134]
 [-1.0008131 -1.4859128]] -> shape: (10, 2)>

In [22]:
prediction.child.decode()

array([[ -9138.91601562, -32360.22460938],
       [ -2589.08642578, -29962.58398438],
       [ -4006.70947266, -27737.55664062],
       [  -669.7623291 , -28234.01367188],
       [   549.5892334 , -31208.20117188],
       [ -6860.75634766, -26845.73632812],
       [  5393.24658203, -29911.54492188],
       [  1014.99353027, -34367.3203125 ],
       [   647.54040527, -25028.42382812],
       [  1125.26403809, -30544.484375  ]])

In [11]:
conv_layer = Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2)

In [12]:
x.child.decode().shape

(10, 3, 50, 50)

In [13]:
conv_layer(x).shape

(10, 32, 52, 52)

In [14]:
bn = torch.nn.BatchNorm2d(3)

In [15]:
output = bn(torch.ones(10, 3, 5, 5))

In [16]:
output.shape

torch.Size([10, 3, 5, 5])

In [17]:
type(output)

torch.Tensor

In [18]:
output.grad_fn()

TypeError: expected 1 arguments, got 0 instead